# Setup

In [21]:
source("/home/vickydiliscia/dmeyf2025/src/churn/5002/config.r")

In [ ]:
# Cargo el index (que a su vez definirá la función)
source(file.path(PATH_UTILS, "index.R"))

# Ejecuto la función que carga todos los utils
source_utils(PATH_UTILS)

# Parametros

## Parametros Experimento

In [3]:
## Semillas
PARAM <- list()
PARAM$semilla_primigenia <- SEEDS[1]

# training y future
PARAM$train <- c(202102)
PARAM$train_final <- c(202101,202102,202103,202104)
PARAM$future <- c(202104)
PARAM$future_final <- c(202106)
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

# undersampling
PARAM$trainingstrategy$undersampling <- 0.1

# carpeta de trabajo
setwd(PATH_THIS_EXPERIMENT)

## Parametros LGBM

In [4]:
PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)

# Librerias

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

# Dataset

In [25]:
# lectura del dataset
dataset <- fread(DATASET_FEATURES, stringsAsFactors= TRUE)

# Tomo hiperparametros

In [10]:
ls()

[1] "agregar_clase_ternaria"     "agregar_lags_dataset"      
 [3] "armar_archivo_envios"       "comparar_cortes"           
 [5] "comparar_semillas"          "DATASET_CLASE"             
 [7] "DATASET_CRUDO"              "DATASET_FEATURES"          
 [9] "ensamblar_promedio"         "entrenar_modelos"          
[11] "entrenar_modelos_paralelos" "enviar_archivo"            
[13] "EXPERIMENT"                 "EXPERIMENT_FOLDER"         
[15] "kbayesiana"                 "LAGS"                      
[17] "N_CLUSTERS"                 "PARAM"                     
[19] "particionar"                "PATH_BASE"                 
[21] "PATH_BO"                    "PATH_BUCKET"               
[23] "PATH_CHURN"                 "PATH_DATASETS"             
[25] "PATH_EXP_CODE"              "PATH_EXPERIMENTS"          
[27] "PATH_KAGGLE"                "PATH_MODELS"               
[29] "PATH_OUTPUTS"               "PATH_PREDICTION"           
[31] "PATH_THIS_EXPERIMENT"       "PATH_UTILS"                
[33] "percentilizar_variables"    "plot_cortes"               
[35] "plot_semillas"              "realidad_evaluar"          
[37] "realidad_inicializar"       "SEEDS"                     
[39] "source_utils"               "TAG"                       
[41] "UNDERSAMPLING"              "us_pct"                    
[43] "USER"

In [6]:
setwd(PATH_BO)

In [11]:
kbayesiana <- "bayesiana.RDATA"  # tu archivo con el opt.state adentro
# Cargar el archivo (esto mete en memoria los objetos guardados, p. ej. opt.state)
load(kbayesiana)

# Si el archivo contiene el estado de la optimización:
bayesiana_salida <- list(opt.path = opt.state$opt.path)
                         
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

PARAM$out$lgbm$y <- tb_bayesiana[1, y]

write_yaml( PARAM, file="PARAM.yml")

print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

[1] "num_iterations"   "learning_rate"    "feature_fraction" "num_leaves"      
 [5] "min_data_in_leaf" "y"                "dob"              "eol"             
 [9] "error.message"    "exec.time"        "ei"               "error.model"     
[13] "train.time"       "prop.type"        "propose.time"     "se"              
[17] "mean"

   num_iterations learning_rate feature_fraction num_leaves min_data_in_leaf
            <int>         <num>            <num>      <int>            <int>
1:           1673     0.0226382        0.3068103        475                1
[1] 0.9437737


# Final train

In [ ]:
## Dataset para el final train

In [22]:
PATH_MODELS

[1] "/home/vickydiliscia/buckets/b1/exp/EXP5002/models"

In [23]:
setwd(PATH_THIS_EXPERIMENT)
models <- "models"
dir.create(models, showWarnings= FALSE)
setwd(PATH_MODELS)

In [26]:
# clase01 - me daba error con los max_bin
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

clase_ternaria,N
<fct>,<int>
CONTINUA,642824
BAJA+2,3938
BAJA+1,3447


## Hiperparametros para el final train

In [27]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 100271

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$max_bin
[1] 31

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 1673

$learning_rate
[1] 0.0226382

$feature_fraction
[1] 0.3068103

$num_leaves
[1] 475

$min_data_in_leaf
[1] 1

In [28]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)
param_normalizado$num_threads <- 8  # no optimizable, solo controla CPU


# entreno LightGBM

modelo_final <- lgb.train(
  data= dtrain_final,
  param= param_normalizado
)

In [29]:
param_normalizado


$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 100271

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$max_bin
[1] 31

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 1673

$learning_rate
[1] 0.0226382

$feature_fraction
[1] 0.3068103

$num_leaves
[1] 475

$min_data_in_leaf
[1] 10

$num_threads
[1] 8

# Training de modelos

In [30]:
result <- entrenar_modelos(
  dtrain  = dtrain_final,
  params  = param_normalizado,
  nrounds = 1375,
  seeds   = SEEDS,
  tag     = TAG
)
# paths de modelos guardados:
sapply(result, function(x) x$path)


Entrenando modelo con semilla 100271 ...

Modelo guardado: /home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed100271.txt

Entrenando modelo con semilla 250061 ...

Modelo guardado: /home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed250061.txt

Entrenando modelo con semilla 345803 ...

Modelo guardado: /home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed345803.txt

Entrenando modelo con semilla 567983 ...

Modelo guardado: /home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed567983.txt

Entrenando modelo con semilla 890009 ...

Modelo guardado: /home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed890009.txt

Entrenando modelo con semilla 910037 ...

Modelo guardado: /home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed910037.txt

Entrenando modelo con semilla 970019 ...

Modelo guardado: /home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_50

100271 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed100271.txt" 
                                                                                  250061 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed250061.txt" 
                                                                                  345803 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed345803.txt" 
                                                                                  567983 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed567983.txt" 
                                                                                  890009 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed890009.txt" 
                                                                                  910037 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed910037.txt" 
                                                                                  970019 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed970019.txt" 
                                                                                  990137 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed990137.txt" 
                                                                                  860689 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed860689.txt" 
                                                                                  780079 
"/home/vickydiliscia/buckets/b1/exp/EXP5002/models/modelo_5002_lag2_us10_seed780079.txt"

In [31]:
model_paths <- list.files(pattern = paste0("^modelo_", TAG, "_seed.*\\.txt$"))
modelos <- lapply(model_paths, lgb.load)

In [ ]:
ensamblar_promedio (
  dataset       = dataset,
  PARAM         = PARAM,
  campos_buenos = campos_buenos,
  experimento   = TAG,
  models_dir    = ".",
  pattern       = paste0("^modelo_", TAG, "_seed.*\\.txt$"),
  out_dir       = "predicciones"
)


# Envios a Kaggle

In [ ]:

# 0) Config (por si no quedó en memoria)
pred_file <- paste0(prediccion",TAG) # tu archivo .txt

# 1) Cargar predicciones desde el archivo
tb_prediccion <- fread(pred_file, sep = "\t")  
# Espera columnas: numero_de_cliente, foto_mes, prob

In [ ]:
tb_prediccion